In [7]:
import pandas as pd

In [8]:
data = pd.read_sql_table("flashcards", "sqlite:///flashcard_ai.db")
data.head()

,Question_number,Has Preamble,Preamble Text,Question,Question has figure,Question figure name,Answer has figure,Answer figure name,Answer,calculations present,...,year,round,due,stability,difficulty,elapsed_days,scheduled_days,reps,lapses,state
0,1,Yes,Solve for $x$ in the equation,$3 \log x+\log 3=\log 81$,No,None,No,None,$\mathrm{x}=3 \quad\left[\log 3 \mathrm{x}^{3}...,Yes,...,2013,Round 1,NaT,0,0,0,0,0,0,None
1,2,Yes,Solve for $x$ in the equation,$2 \log 4+\log x=\log 64$,No,None,No,None,$\mathrm{x}=4 \quad[\log 16 \mathrm{x}=\log 64...,Yes,...,2013,Round 1,NaT,0,0,0,0,0,0,None
2,3,Yes,Solve for $x$ in the equation,$\log 36-2 \log x=\log 4$,No,None,No,None,$\mathrm{x}=3 \quad\left[\log (36 / 4)=\log ^{...,Yes,...,2013,Round 1,NaT,0,0,0,0,0,0,None
3,4,Yes,Planck's constant is $6.63 \times 10^{-34} \ma...,What is the energy of a photon of frequency $5...,No,None,No,None,$3.58 \times 10^{-19} \mathrm{~J} 3.58 \times ...,Yes,...,2013,Round 1,NaT,0,0,0,0,0,0,None
4,5,Yes,Planck's constant is $6.63 \times 10^{-34} \ma...,What is the momentum of a photon of frequency ...,No,None,No,None,$1.19 \times 10^{-27} \mathrm{~kg} \cdot \math...,Yes,...,2013,Round 1,NaT,0,0,0,0,0,0,None


In [9]:
data.iloc[14701]

Question_number                                                         5
Has Preamble                                                           No
Preamble Text                                                        None
Question                What is the colour of the pupa of a swallow ta...
Question has figure                                                    No
Question figure name                                                 None
Answer has figure                                                      No
Answer figure name                                                   None
Answer                  The pupa is brown with green spots. This blend...
calculations present                                                   No
Subject                                                           Biology
year                                                                 2009
round                                                             Round 1
due                                   

In [4]:
import sqlite3

try:

    # Making a connection between sqlite3
    # database and Python Program
    sqliteConnection = sqlite3.connect("flashcard_ai.db")

    # If sqlite3 makes a connection with python
    # program then it will print "Connected to SQLite"
    # Otherwise it will show errors
    print("Connected to SQLite")

    # Getting all tables from sqlite_master
    sql_query = """SELECT name FROM sqlite_master 
    WHERE type='table';"""

    # Creating cursor object using connection object
    cursor = sqliteConnection.cursor()

    # executing our sql query
    cursor.execute(sql_query)
    print("List of tables\n")

    # printing all tables list
    print(cursor.fetchall())

except sqlite3.Error as error:
    print("Failed to execute the above query", error)

finally:

    # Inside Finally Block, If connection is
    # open, we need to close it
    if sqliteConnection:

        # using close() method, we will close
        # the connection
        sqliteConnection.close()

        # After closing connection object, we
        # will print "the sqlite connection is
        # closed"
        print("the sqlite connection is closed")

Connected to SQLite
List of tables

[('flashcards',)]
the sqlite connection is closed


In [5]:
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error
from contextlib import contextmanager

DB_NAME = "flashcard_ai.db"


@contextmanager
def get_db():
    # create db connection
    try:
        conn = sqlite3.connect(DB_NAME)
        yield conn
    except Error as e:
        raise
    finally:
        if conn:
            conn.close()


def select_all_flashcards():
    with get_db() as conn:
        cur = conn.cursor()
        try:
            cur.execute("SELECT * FROM flashcards")
            return cur.fetchall()
        except Error as e:
            raise

In [11]:
len(select_all_flashcards())

32315

In [2]:
import os
from langchain_groq import ChatGroq

# from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
# load_dotenv(find_dotenv())

##Using Groq model
# GROQ_API_KEY = os.getenv("GROQ_API_KEY")

GROQ_API_KEY = "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"

model = ChatGroq(model="llama3-8b-8192", api_key=GROQ_API_KEY)


## Prompt template
def get_explanation(question, answer):
    template = """You are a helpful scientist and mathematician assistant. You explain scientific and mathematical answers to questions
    to the best of your ability in a clear and concise manner maintaining a scientific and educational tone.
    If you don't know the answer, simply state that you don't know instead of attempting to fabricate a response.
    User question: {user_question}
    Answer: {answer}
"""
    prompt = ChatPromptTemplate.from_template(template)

    llm = model

    chain = prompt | llm | StrOutputParser()

    return chain.invoke({"user_question": question, "answer": answer})

In [6]:
question = """
What is the colour of the pupa of a swallow tail citrus butterfly and what is the importance of this colouration?
"""


answer = """
The pupa is brown with green spots. This blends with the surroundings; protecting it from predators (this is called camouflage).
  """


explanation = get_explanation(question, answer)

print(explanation)

Thank you for asking!

The pupa of the Swallowtail Citrus butterfly, specifically the Papilio xenogamius, has a distinctive coloration. According to scientific studies and observations, the pupa is typically brown with green spots.

This remarkable coloration serves an important purpose: it provides camouflage. The brown and green coloration allows the pupa to blend seamlessly with its surroundings, making it less visible to potential predators. This phenomenon is known as "cryptic coloration" or "camouflage."

By blending in with its environment, the pupa reduces its chances of being detected and attacked by predators, increasing its chances of survival until it emerges as a fully formed adult butterfly.

So, in summary, the brown with green spots coloration of the pupa of the Swallowtail Citrus butterfly is a remarkable example of adaptation, allowing it to conceal itself and avoid predation.

Would you like to know more about butterfly biology or camouflage in general?


In [5]:
type(explanation)

str